## Filling out the damn NSF Conflict of interest form ##

The NSF has a form for identifying potential conflicts of interest for the very laudable reason of not having your best friend and collaborator review you grant proposal (or more specifically, not having your competitor's best friend and collaborator review theirs and your proposal on the same panel). 
They take it very seriously in the best possible way, and this is one of many examples of government working hard and succeeding at doing the right thing at low cost.
Unfortunately the form requires co-authors and their affiliations on all publications for the last 4 years, which for me and other people is a bit of a pain.
As a person on a number of genome papers, I (currently, who knows about the future) have a lot of co-authors. Others can have fewer co-authors per paper, but may be more prolific in publication. - My worst case example here is Botanist [Hans Lambers](https://scholar.google.com/citations?hl=en&user=up0v5soAAAAJ&view_op=list_works&sortby=pubdate), But I am also not sure about physicists and chemists, or others outside of pubmed.

### My aim here is to automate the filling out of this form. ### 
Most researchers have no trouble finding the last 4 years of references, or even getting all of the co-authors with a little screwing around in excel, but getting all of the affliations for those people requires getting out of the reference manager software, and cutting and pasting from the pdfs.
Pubmed does keep affliations for all authors, so I am going to try that first.
Other options include Thomson Reuters Web of Science which also has affliation information that can be scraped.

_Possible inputs to this script could be:_ 

1. a list of pubmed id's  CAN BE DOWNLOADED FROM NCBI USING THE **send to** button, which gives a file and options for xml, pmids ect. - WORKS GREAT!
2. a link to a NCBI biography - e.g. https://www.ncbi.nlm.nih.gov/myncbi/1hQ1zEK_eulQ8/bibliography/public/  THIS DOESN"T WORK BECAUSE OF PRETTY FORMATTING  
3. a link to a google scholar profile - for example: https://scholar.google.com/citations?user=EPC9uVAAAAAJ&hl=en THIS JUST DOESN'T WORK - NO AFFILIATIONS

And as of right now the NSF form in question is [here](https://www.nsf.gov/bfa/dias/policy/coa/coa_template.xlsx) and the instructions are [here](https://www.nsf.gov/bfa/dias/policy/coa/faqs_coatemplatemay18.pdf).

# Instructions for using pubmed to get a list of pmids
1. go to pubmed and search for your name in this manner  
__Brown SJ[Author] AND Kansas [affiliation] AND ( "2015/05/31"[PDat] : "2019/05/31"[PDat] )__  
Do that search here: https://www.ncbi.nlm.nih.gov/pubmed/
but change the name to your name and initials, change the dates for the last 4 years,  
and put a word in the affilation to rule out other people with the same name (in this case Susan J Brown is at Kansas State University)  

2. Check the results from the search, - make sure it has everything you want in the results, if not _play with it a bit!_  
__remember that Pubmed doesn't do everything, and there may be papers you have to add manually__

3. Save the results from that search as a list of PMIDs:
    - click on "Send to", For "Choose Destination" pick "File", For "Format" pick "PMID List", and last "Create File"
    - This will download a text file to your computer.  
  
  
4. __Edit the pmid_list_text_file location in the cell below.__  

5. Run all of the cells 1 by 1, note you may need to install biopython, pandas and a bunch of other stuff.  
The output is a .xlsx file called "Paste_me_into_NSF_coa_template_table_4.xlsx" created in the working directory.
From this file you will have to manaually increase the table 4 size in the NSF excel tenplate, and paste in your co-authors and affiliations  
If you have more than ~20 co-authors in the last 4 years this will save you some time

6. __GOOD LUCK!__

In [1]:
# PLEASE EDIT THIS PATH TO POINT TO THE PMID LIST 
pmid_list_text_file = "pubmed_result-5.txt"

In [9]:
#install biopython with conda? - YES - conda install biopython
from Bio import Entrez 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
 # this code from https://marcobonzanini.com/2015/01/12/searching-pubmed-with-python/ and modified by me
def fetch_ref(reference_id): # this needs some error checking
    Entrez.email = 'srrichards@ucdavis.edu'
    handle = Entrez.efetch(db='pubmed', retmode='xml', id=reference_id)
    results = Entrez.read(handle)
    return results

In [11]:
# and test this with a PMID number
#my_results = fetch("29686065")
my_results = fetch_ref("29602370")
#print(my_results['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
for author in my_results['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']:
    #print(author.keys()) # This was a real pain in the ass
    #print(author['LastName'], author['ForeName'])
    
    affil_string = None
    for affils in author['AffiliationInfo']:
        if affil_string:
            affil_string = affil_string + "; " + affils['Affiliation']
        else:
            affil_string = affils['Affiliation']
        #print(affils['Affiliation'])
    print(author['LastName'], ", ", author['ForeName'], ", ", affil_string, "\n", sep="")

Richards, Stephen, Human Genome Sequencing Center, Baylor College of Medicine, Houston, TX 77030, United States.

Childers, Anna, USDA-ARS, Bee Research Laboratory, Beltsville, MD, United States.

Childers, Christopher, USDA-ARS, National Agricultural Library, Knowledge Services Division, United States.



In [12]:
### Making this into a function that takes a PMID number and returns a list of Name (last, first), and affliation (not split into departmewnt.)
### Note this could get the date and check that it was published in the last 4 years

def get_coauthors_and_affils(pmid):
    '''
    This function takes a pmid for a paper, and returns a list of dictionaries for each co-author with the co-authors name and affiliation
    for example it returns a list of 
    {'co_author':'Richards, Stephen', 'affil':'Human Genome Sequencing Center, Baylor College of Medicine, Houston, TX 77030, United States.'} things.
    
    It calls the function fetch_ref above - all this bit does is parse the returned reference from entrez
    '''
    my_results = fetch_ref(pmid)
    author_list = []
    for author in my_results['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']:
        #print(author.keys()) # This was a real pain in the ass
        #print(author['LastName'], author['ForeName'])
        affil_string = None
        for affils in author['AffiliationInfo']:
            if affil_string:
                affil_string = affil_string + "; " + affils['Affiliation']
            else:
                affil_string = affils['Affiliation']
            #print(affils['Affiliation'])
        author_string = author['LastName'] + ", " + author['ForeName']
        #print(author_string, ", ", affil_string, "\n", sep="")
        # OR RETURN A Pandas Df, and print that to a fresh .xlsx file
        author_list.append({"co_author":author_string, "affil":affil_string})
    return(author_list)
        
co_author_and_affils_list = get_coauthors_and_affils("29602370")
print(co_author_and_affils_list)

[{'co_author': 'Richards, Stephen', 'affil': 'Human Genome Sequencing Center, Baylor College of Medicine, Houston, TX 77030, United States.'}, {'co_author': 'Childers, Anna', 'affil': 'USDA-ARS, Bee Research Laboratory, Beltsville, MD, United States.'}, {'co_author': 'Childers, Christopher', 'affil': 'USDA-ARS, National Agricultural Library, Knowledge Services Division, United States.'}]


In [13]:
# send a list of PMIDs (called accession numbers in EndNote) to get_coauthors_and_affils to see how it goes:
# also note - if I was a cool person I would get this list from NCBI biography, or Google Scholar

def pmid_list_to_co_authors_and_affils(pmid_list):
    
    Seen_co_author_before = [] # because the cool people will check for duplicates and order alphabeticaly - NOTE NOT PERFECT ON DUPLICATES BUT A GOOD START
    co_authors_list_to_sort_and_print = []

    all_co_authors = []
    for pmid in pmid_list:
        all_co_authors.append(get_coauthors_and_affils(pmid))
    
    for co_author in all_co_authors:
        for items in co_author:
            if items['co_author'] in Seen_co_author_before:
                #print("skip a repeat name")
                pass
            else:
                #print(items['co_author'], items['affil'], "\n")
                co_authors_list_to_sort_and_print.append({'co_author':items['co_author'], 'affil':items['affil']})
                Seen_co_author_before.append(items['co_author'])
  
    # print out a sorted co_author list - THIS NEEDS TO BE PRINTED OUT TO EXCEL FOR THE FORM
    number_of_co_authors = len(co_authors_list_to_sort_and_print)
    print("number of coauthors semi deduplicated = ", number_of_co_authors, "\n")

    from operator import itemgetter 
    df = pd.DataFrame(co_authors_list_to_sort_and_print)
    df = df.sort_values(by=['co_author'])
    df['tag'] = 'A:'
    df = df[['tag','co_author', 'affil']]
 
    return(df)
    
## NOTE I COULD GO A BIT FUTHER AND ASK IF NAMES NEXT TO EACH OTHER IN THE SORTED LIST ARE VERY SIMILAR ###
# test lists
PMID_list_Richards_2018 =[30626321, 29602357, 29386578, 29602370, 29689052, 29634279, 30935422, 29776407, 29686065, 29403074, 29998472, 30463532]
PMID_list_Richards_2015_2019 = [30626321, 29602357, 29386578, 29602370, 29689052, 29634279, 30935422, 29776407, 29686065, 29403074, 29998472, 30463532, 28017728, 28756775, 29220441, 28756777, 28209133, 29055159, 28460028, 28942770, 26439791, 27436555, 27659211, 27189996, 27832824, 27194801, 27617305, 27522922, 26951779, 27401754, 26836814, 26528622, 26944522, 25660540, 26580012, 25908251, 26085980, 26003218, 26051890, 26108605]
PMID_list_Sue_Brown_2015_2019 = [30943207,30820572, 30630105, 30576522, 30414108, 29959091, 29220441, 29145861, 28973882, 28791475, 27522922, 26454014, 26416786]
PMID_list_small_test = [29602357]

# excel file
excel_file = "/Users/stephenr/Programing/NSF_COA_Form_files/coa_template.xlsx"
# test the funciton:

df = pmid_list_to_co_authors_and_affils(PMID_list_small_test)
df

number of coauthors semi deduplicated =  2 



,tag,co_author,affil
1,A:,"Richards, Stephen","Human Genome Sequencing Center, Department of ..."
0,A:,"Wiegmann, Brian M","Department of Entomology & Plant Pathology, No..."


### After alot of screwing around, I have given up on inserting rows into the NSF table, and instead will just create a new .xlsx file using Pandas 
### For the user to paste into the NSF coa template

In [15]:
# this cell takes a xml file and puts the data into the right Excel file place.
#pmid_list_text_file = "/Users/stephenr/Programing/NSF_COA_Form_files/SueBrown_pmid_list_pubmed_result-4.txt"
# pubmed_xml_file = "/Users/stephenr/Programing/NSF_COA_Form_files/pubmed_result.xml" # just goig with what works for now.
fh = open(pmid_list_text_file, mode='r')
pmid_list = []  # have to screw around to get the text file with line returns into a nice clean list for the function
for pmid in fh.readlines():
    pmid = pmid.rstrip()
    #print(pmid, sep="")
    pmid_list.append(pmid)
#pmid_list
df = pmid_list_to_co_authors_and_affils(pmid_list)
df.to_excel('Paste_me_into_NSF_coa_template_table_4.xlsx')
df.head()

number of coauthors semi deduplicated =  89 



,tag,co_author,affil
2,A:,"Alkan, Zeynep",USDA ARS Western Human Nutrition Research Cent...
1,A:,"Ardeshir, Amir","California National Primate Research Center, U..."
51,A:,"Arikit, Siwaret","Delaware Biotechnology Institute, University o..."
85,A:,"Carter, Amanda","Department of Molecular and Cellular Biology, ..."
10,A:,"Cheng, Hans","USDA-ARS, Avian Disease and Oncology Laborator..."
